In [1]:
import polars as pl
import polars.selectors as cs

import numpy as np
import datetime  

from sklearn import preprocessing
from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.model_selection import train_test_split

from functools import lru_cache

import gc

In [2]:
#%%time
#pl.read_parquet("data/transactions_?.parquet").describe()

In [3]:
#pl.scan_parquet("data/transactions_1.parquet").collect().n_unique()

In [4]:
delweek=93
delweek=105
delweek=102
#delweek=103

In [5]:
"""
@lru_cache
def lenmonth(date)->int:
    m=date.month()
    if m in [1,3,5,7,8,10,12]:
        return 31
    elif m==2:
        if y%4==0:
            return 29
        else:
            return 28
    else:
        return 30    
        """
0

0

In [6]:
workdays={}
d1 = datetime.date(2022, 1, 1)
while d1<datetime.date(2025, 1, 1):
    #print(d1)
    #workdays.append([d1,0 if d1.weekday() in [5,6] else 1 ] )
    workdays[d1] = 0 if d1.weekday() in [5,6] else 1
    
    d1=d1+datetime.timedelta(1) 
workdays[datetime.date(2022, 1, 3)] = 0
workdays[datetime.date(2022, 1, 4)] = 0
workdays[datetime.date(2022, 1, 5)] = 0
workdays[datetime.date(2022, 1, 6)] = 0
workdays[datetime.date(2022, 1, 7)] = 0

workdays[datetime.date(2022, 2, 23)] = 0

workdays[datetime.date(2022, 3, 5)] = 1
workdays[datetime.date(2022, 3, 7)] = 0
workdays[datetime.date(2022, 3, 8)] = 0

workdays[datetime.date(2022, 5, 2)] = 0
workdays[datetime.date(2022, 5, 3)] = 0
workdays[datetime.date(2022, 5, 9)] = 0
workdays[datetime.date(2022, 5, 10)] = 0

workdays[datetime.date(2022, 6, 13)] = 0

workdays[datetime.date(2022, 11, 4)] = 0
########################################## 2023
workdays[datetime.date(2023, 1, 2)] = 0
workdays[datetime.date(2023, 1, 3)] = 0
workdays[datetime.date(2023, 1, 4)] = 0
workdays[datetime.date(2023, 1, 5)] = 0
workdays[datetime.date(2023, 1, 6)] = 0

workdays[datetime.date(2023, 2, 23)] = 0
workdays[datetime.date(2023, 2, 24)] = 0

workdays[datetime.date(2023, 3, 8)] = 0

workdays[datetime.date(2023, 5, 1)] = 0
workdays[datetime.date(2023, 5, 8)] = 0
workdays[datetime.date(2023, 5, 9)] = 0

workdays[datetime.date(2023, 6, 12)] = 0

workdays[datetime.date(2023, 11, 6)] = 0
########################################## 2024
workdays[datetime.date(2024, 1, 1)] = 0
workdays[datetime.date(2024, 1, 2)] = 0
workdays[datetime.date(2024, 1, 3)] = 0
workdays[datetime.date(2024, 1, 6)] = 0
workdays[datetime.date(2024, 1, 7)] = 0
workdays[datetime.date(2024, 1, 8)] = 0

workdays[datetime.date(2024, 5, 1)] = 0
workdays[datetime.date(2024, 5, 2)] = 0
workdays[datetime.date(2024, 5, 8)] = 0
workdays[datetime.date(2024, 5, 9)] = 0

workdays[datetime.date(2024, 6, 12)] = 0
workdays[datetime.date(2024, 6, 13)] = 0

workdays[datetime.date(2024, 11, 1)] = 1
workdays[datetime.date(2024, 11, 3)] = 0
workdays[datetime.date(2024, 11, 4)] = 0

workdays[datetime.date(2024, 12, 31)] = 0

workdays
"""
calendar000 = pl.read_csv("data/calendar.csv", try_parse_dates=True)
calendar000.with_columns(
                      (pl.col("date")+datetime.timedelta(-5)).replace(workdays).cast(pl.Int8).alias("work_5"), 
                      (pl.col("date")+datetime.timedelta(-4)).replace(workdays).cast(pl.Int8).alias("work_4"), 
                      (pl.col("date")+datetime.timedelta(-3)).replace(workdays).cast(pl.Int8).alias("work_3"), 
                      (pl.col("date")+datetime.timedelta(-2)).replace(workdays).cast(pl.Int8).alias("work_2"), 
                      (pl.col("date")+datetime.timedelta(-1)).replace(workdays).cast(pl.Int8).alias("work_1"), 
                      pl.col("date").replace(workdays).cast(pl.Int8).alias("work"),
                      (pl.col("date")+datetime.timedelta(1)).replace(workdays).cast(pl.Int8).alias("work1"), 
                      (pl.col("date")+datetime.timedelta(2)).replace(workdays).cast(pl.Int8).alias("work2"), 
                      (pl.col("date")+datetime.timedelta(3)).replace(workdays).cast(pl.Int8).alias("work3"), 
                      (pl.col("date")+datetime.timedelta(4)).replace(workdays).cast(pl.Int8).alias("work4"), 
                      (pl.col("date")+datetime.timedelta(5)).replace(workdays).cast(pl.Int8).alias("work5"), 
                     ).with_columns( 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")+pl.col("work5")).alias("w5_after"), 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")).alias("w4_after"), 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")).alias("w3_after"), 
                      (pl.col("work1")+pl.col("work2")).alias("w2_after"), 
                      (pl.col("work_1")+pl.col("work_2")).alias("w2_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")).alias("w3_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")+pl.col("work_4")).alias("w4_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")+pl.col("work_4")+pl.col("work_5")).alias("w5_before"), 
                     )
"""
workdays

{datetime.date(2022, 1, 1): 0,
 datetime.date(2022, 1, 2): 0,
 datetime.date(2022, 1, 3): 0,
 datetime.date(2022, 1, 4): 0,
 datetime.date(2022, 1, 5): 0,
 datetime.date(2022, 1, 6): 0,
 datetime.date(2022, 1, 7): 0,
 datetime.date(2022, 1, 8): 0,
 datetime.date(2022, 1, 9): 0,
 datetime.date(2022, 1, 10): 1,
 datetime.date(2022, 1, 11): 1,
 datetime.date(2022, 1, 12): 1,
 datetime.date(2022, 1, 13): 1,
 datetime.date(2022, 1, 14): 1,
 datetime.date(2022, 1, 15): 0,
 datetime.date(2022, 1, 16): 0,
 datetime.date(2022, 1, 17): 1,
 datetime.date(2022, 1, 18): 1,
 datetime.date(2022, 1, 19): 1,
 datetime.date(2022, 1, 20): 1,
 datetime.date(2022, 1, 21): 1,
 datetime.date(2022, 1, 22): 0,
 datetime.date(2022, 1, 23): 0,
 datetime.date(2022, 1, 24): 1,
 datetime.date(2022, 1, 25): 1,
 datetime.date(2022, 1, 26): 1,
 datetime.date(2022, 1, 27): 1,
 datetime.date(2022, 1, 28): 1,
 datetime.date(2022, 1, 29): 0,
 datetime.date(2022, 1, 30): 0,
 datetime.date(2022, 1, 31): 1,
 datetime.date(20

In [7]:
#%%time
#inns = pl.read_csv("data/sample_submit.csv"
#           )["inn_id"].unique()
#inns

#notworks=pl.scan_parquet("data/transactions_?.parquet"
#               ).filter(pl.col("doc_payer_inn").is_in(inns)
#               ).filter(pl.col("doc_payer_bank_name_flag")==1
#                       ).with_columns(
#                            pl.col("date").dt.date(),
#                            pl.col("date").replace(workdays).cast(pl.Int8).alias("work"),
#                        ).filter(pl.col("work")==0
#                                ).group_by("doc_payer_inn"
#                                          ).agg(pl.col("date").n_unique()                           
#                        ).collect()
#notworks.to_pandas()["date"].hist(bins=240)

In [8]:
%%time
"""
innstat000 = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                            ).filter( pl.col("week")<delweek 
                            ).filter( pl.col("target")>0 
                            ).with_columns(                              
                                (pl.col("week")-pl.col("week").shift(1).over("inn_id")).alias("lbreak"),
                                    ).group_by("inn_id"
                                              ).agg((delweek-pl.col("week").max()-1).alias("endbreak"),
                                                    (pl.col("week").max()-pl.col("week").min()+1).alias("length"),
                                                    #pl.col("week").min().alias("startweek"),
                                                    pl.col("week").count().alias("countpositive"),
                                                    pl.col("lbreak").max().alias("maxlbreak"),
                                                   ).with_columns((pl.col("length")-pl.col("countpositive")).alias("countzero") 
                                                                 )
innstat000#.max()
"""
0

CPU times: user 14 μs, sys: 2 μs, total: 16 μs
Wall time: 19.1 μs


0

In [9]:
inns = pl.read_csv("data/sample_submit.csv"
           )["inn_id"].unique()
inns

inns_ip = pl.read_parquet("data/profiles.parquet"
           ).filter(pl.col("ipul")=="ip")["inn_id"].unique()
inns,inns_ip

(shape: (51_963,)
 Series: 'inn_id' [str]
 [
 	"inn1882781"
 	"inn3365634"
 	"inn3268019"
 	"inn110614"
 	"inn962316"
 	…
 	"inn128047"
 	"inn584148"
 	"inn3269390"
 	"inn4235000"
 	"inn1986045"
 ],
 shape: (1_434_475,)
 Series: 'inn_id' [str]
 [
 	"inn3974108"
 	"inn2358679"
 	"inn3216028"
 	"inn2715643"
 	"inn1045025"
 	…
 	"inn3428840"
 	"inn3434091"
 	"inn4914478"
 	"inn369700"
 	"inn3249095"
 ])

In [10]:
%%time
#расходы из ВТБ
out_vtb=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payer_inn").is_in(inns)
               ).filter(pl.col("doc_payer_bank_name_flag")==1
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payer_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   pl.col("date").n_unique().cast(pl.UInt8),
                                                   pl.col("doc_payee_inn").n_unique().cast(pl.UInt16).alias("cinn"),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payer_bank_name_encoded").n_unique(),
                                            ).with_columns(                              
                                                np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),
                                                    ).rename(lambda column_name: ("" if column_name in ["doc_payer_inn","week"] else "out_vtb_") + column_name
                                                          ).collect()
out_vtb

CPU times: user 1min 9s, sys: 15 s, total: 1min 24s
Wall time: 24.3 s


doc_payer_inn,week,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log
str,i8,f64,f64,u8,u16,f64
"""inn1719815""",78,2.0,84052.357422,1,2,11.339207
"""inn5114042""",38,3.0,21992.11499,3,3,9.998485
"""inn2730564""",90,13.0,275410.529541,5,12,12.526022
"""inn4702804""",46,12.0,1.2675e6,4,4,14.052567
"""inn3789252""",29,10.0,690921.996094,2,4,13.445784
…,…,…,…,…,…,…
"""inn3993325""",39,2.0,2111.028198,2,2,7.655404
"""inn2925954""",103,5.0,1.9352e6,3,4,14.475731
"""inn1416762""",23,4.0,50560.746094,1,1,10.830951


In [11]:
%%time
#расходы из ВТБ на ИП
out_vtbip=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payer_inn").is_in(inns)
               ).filter(pl.col("doc_payer_bank_name_flag")==1
               ).filter(pl.col("doc_payee_inn").is_in(inns_ip)
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payer_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   pl.col("doc_payee_inn").n_unique().cast(pl.UInt16).alias("cinn"),
                                                   #pl.col("date").n_unique().cast(pl.UInt8),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payer_bank_name_encoded").n_unique(),
                                            #).with_columns(                              
                                            #    np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),
                                                    ).rename(lambda column_name: ("" if column_name in ["doc_payer_inn","week"] else "out_vtbip_") + column_name
                                                          ).collect()
out_vtbip

CPU times: user 1min 29s, sys: 13.2 s, total: 1min 42s
Wall time: 26.9 s


doc_payer_inn,week,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn
str,i8,f64,f64,u16
"""inn2543844""",65,7.0,95243.117188,4
"""inn345058""",6,4.0,31341.775635,4
"""inn4537201""",90,4.0,14481.200439,4
"""inn4063205""",40,6.0,564205.671875,2
"""inn3437932""",53,1.0,18379.654297,1
…,…,…,…,…
"""inn4404902""",4,1.0,32649.064453,1
"""inn4408230""",34,11.0,949196.712524,6
"""inn3484689""",18,2.0,60291.005859,2


In [12]:
"""
%%time
#расходы не из ВТБ
out_notvtb=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payer_inn").is_in(inns)
               ).filter(pl.col("doc_payer_bank_name_flag")==0
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payer_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   #pl.col("date").n_unique(),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payer_bank_name_encoded").n_unique(),
                                            #).with_columns(                              
                                            #    np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),
                                                  ).rename(lambda column_name: ("" if column_name in ["doc_payer_inn","week"] else "out_notvtb_") + column_name
                                                  ).collect()
out_notvtb
"""
0

0

In [13]:
%%time
#приходы в ВТБ
in_vtb=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payee_inn").is_in(inns)
               ).filter(pl.col("doc_payee_bank_name_flag")==1
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payee_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   pl.col("date").n_unique().cast(pl.UInt8),
                                                   pl.col("doc_payer_inn").n_unique().cast(pl.UInt16).alias("cinn"),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payee_bank_name_encoded").n_unique(),
                                            ).with_columns(                              
                                                np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),                                                   
                                                  ).rename(lambda column_name: ("" if column_name in ["doc_payee_inn","week"] else "in_vtb_") + column_name
                                                          ).collect()
in_vtb

CPU times: user 1min 42s, sys: 18.5 s, total: 2min 1s
Wall time: 35.3 s


doc_payee_inn,week,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log
str,i8,f64,f64,u8,u16,f64
"""inn2485082""",44,21.0,5.3701e6,6,13,15.496354
"""inn3809980""",81,19.0,685782.127319,6,5,13.438317
"""inn4637155""",35,1.0,72626.195312,1,1,11.193095
"""inn3000897""",12,13.0,86982.182739,5,7,11.37347
"""inn328203""",30,33.0,1.8342e7,3,9,16.724727
…,…,…,…,…,…,…
"""inn1867514""",76,4.0,236278.439453,3,4,12.37277
"""inn4430422""",3,1.0,12960.824219,1,1,9.469764
"""inn1463221""",4,10.0,2.3530e6,5,2,14.671219


In [14]:
%%time
#приходы в ВТБ от ИП
in_vtbip=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payee_inn").is_in(inns)
               ).filter(pl.col("doc_payee_bank_name_flag")==1
               ).filter(pl.col("doc_payer_inn").is_in(inns_ip)
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payee_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   pl.col("doc_payer_inn").n_unique().cast(pl.UInt16).alias("cinn"),
                                                   #pl.col("date").n_unique().cast(pl.UInt8),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payee_bank_name_encoded").n_unique(),
                                            #).with_columns(                              
                                            #    np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),                                                   
                                                  ).rename(lambda column_name: ("" if column_name in ["doc_payee_inn","week"] else "in_vtbip_") + column_name
                                                          ).collect()
in_vtbip

CPU times: user 1min 41s, sys: 14.7 s, total: 1min 56s
Wall time: 31.9 s


doc_payee_inn,week,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn
str,i8,f64,f64,u16
"""inn1347124""",69,3.0,977330.632812,3
"""inn1624326""",88,18.0,1.1409e6,14
"""inn4466660""",104,3.0,314604.748047,3
"""inn5023869""",60,6.0,414120.78125,1
"""inn2704164""",31,6.0,28938.990723,5
…,…,…,…,…
"""inn2673635""",26,14.0,336608.215942,5
"""inn4616335""",25,1.0,1383.345459,1
"""inn3811017""",20,1.0,5108.200195,1


In [15]:
"""
#приходы не в ВТБ
in_notvtb=pl.scan_parquet("data/transactions_?.parquet"
               ).filter(pl.col("doc_payee_inn").is_in(inns)
               ).filter(pl.col("doc_payee_bank_name_flag")==0
                      ).with_columns(pl.col("date").dt.date()
                             ).join(pl.scan_csv("data/calendar.csv", try_parse_dates=True
                                               ).drop("part"
                                                     ).with_columns(pl.col("week").cast(pl.Int8)),
                                    on="date",
                                    how="left"
                                   ).group_by("doc_payee_inn","week"
                                             ).agg(pl.col("trns_count").sum(),
                                                   pl.col("trns_amount").sum(),
                                                   #pl.col("date").n_unique(),
                                                   #pl.col("trns_class_encoded").n_unique(),
                                                   #pl.col("doc_payee_bank_name_encoded").n_unique(),
                                            #).with_columns(                              
                                            #    np.log1p(pl.col("trns_amount")).alias("trns_amount_log"),                                                   
                                                  ).rename(lambda column_name: ("" if column_name in ["doc_payee_inn","week"] else "in_notvtb_") + column_name
                                                          ).collect()
in_notvtb
"""
0

0

In [16]:
%%time
innweeks = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                #pl.col("target").alias("target_"),
                            ).with_columns(                              
                                pl.col("target").log1p().alias("target"),
                                #pl.col("target").sign().alias("target"),
                            ).with_columns(                              
                                (1-pl.col("target").sign()).cast(pl.Int8).alias("sign"),
    
                            ).join(out_vtb,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payer_inn","week"],
                                   how="left"
                            ).join(out_vtbip,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payer_inn","week"],
                                   how="left"
                            #).join(out_notvtb,
                            #       left_on=["inn_id","week"],
                            #       right_on=["doc_payer_inn","week"],
                            #       how="left"
                            ).join(in_vtb,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payee_inn","week"],
                                   how="left"
                            ).join(in_vtbip,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payee_inn","week"],
                                   how="left"
                            #).join(in_notvtb,
                            #       left_on=["inn_id","week"],
                            #       right_on=["doc_payee_inn","week"],
                            #       how="left"
                            ).fill_null(0
                            ).with_columns(#cs.by_dtype(pl.UInt32).cast(pl.UInt8),
                                           (pl.col("in_vtb_trns_amount")-pl.col("out_vtb_trns_amount")
                                           ).cast(pl.Float32).alias("cum"),
                                           cs.ends_with("_date").cast(pl.UInt8),
                                           cs.ends_with("_class_encoded").cast(pl.UInt8),
                                           cs.ends_with("_name_encoded").cast(pl.UInt16),
                                           cs.ends_with("_trns_count").cast(pl.UInt16),
                                           cs.starts_with("target").cast(pl.Float32),
                                           cs.ends_with("_trns_amount").cast(pl.Float32),
                                           cs.ends_with("_log").cast(pl.Float32),
                                           #cs.by_dtype(pl.Float64).cast(pl.Float32),
                            ).with_columns(
                                           #pl.col("cum").cum_sum().over("inn_id").alias("cumsum"),
                                           (pl.col("out_vtbip_trns_amount")/(pl.col("out_vtb_trns_amount")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_trns_amount")/(pl.col("in_vtb_trns_amount")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("out_vtbip_trns_count")/(pl.col("out_vtb_trns_count")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_trns_count")/(pl.col("in_vtb_trns_count")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("out_vtbip_cinn")/(pl.col("out_vtb_cinn")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_cinn")/(pl.col("in_vtb_cinn")+0.00001)*255
                                           ).cast(pl.UInt8),
                            ).join(pl.read_parquet("innweekstartsum.parquet"),
                                   on=["inn_id","week"],
                                   how="left",
                                  
                            #).with_columns(
                            #               pl.col("cumsum").cum_min().over("inn_id").alias("cummin"),
                            #).with_columns(
                            #               (pl.when(pl.col("cummin")>0
                            #                       ).then(pl.lit(0)
                            #                             ).otherwise(pl.col("cummin"))
                            #               ).alias("cummin"),
                            #).with_columns(
                            #               pl.col("cumsum")-pl.col("cummin"),
                            )#.drop("cummin")
innweeks#.filter( pl.col("target")<5).filter( pl.col("target")>0)#.sum()

CPU times: user 13.6 s, sys: 4.03 s, total: 17.7 s
Wall time: 5.44 s


inn_id,week,target,sign,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn,cum,cumsum
str,i8,f32,i8,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,f32
"""inn1000051""",0,12.95314,0,2,422159.3125,2,2,12.95314,0,0,0,5,36499.808594,3,1,10.50509,0,0,0,-385659.53125,0.0
"""inn1000051""",1,7.392578,0,1,1622.887451,1,1,7.392578,0,0,0,9,643022.5625,3,5,13.373937,56,130,101,641399.6875,13733.0625
"""inn1000051""",2,13.929488,0,1,1.1207e6,1,1,13.929488,0,0,0,9,433354.75,4,3,12.979314,113,238,169,-687370.875,655132.75
"""inn1000051""",3,10.583951,0,1,39494.851562,1,1,10.583951,0,0,0,5,10727.206055,4,1,9.280632,0,0,0,-28767.646484,13069.5
"""inn1000051""",4,12.972155,0,3,430263.25,3,3,12.972155,0,0,0,5,116672.53125,5,2,11.667135,0,0,0,-313590.71875,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999886""",101,15.637834,0,39,6.186212e6,6,22,15.637834,13,1,23,55,2.09658e7,6,36,16.858402,9,4,14,1.4779589e7,8.78596928e8
"""inn999886""",102,17.042799,0,49,2.5211184e7,5,24,17.042799,5,0,10,54,1.6688619e7,6,44,16.630238,0,0,0,-8.522564e6,8.93376576e8
"""inn999886""",103,16.736929,0,61,1.856761e7,5,31,16.736929,41,5,57,28,1.5962664e7,5,25,16.585764,27,26,30,-2.6049e6,8.8485408e8


In [17]:
#innweeks.group_by("inn_id").agg( pl.col("week").count()-pl.col("week").max()+pl.col("week").min() )["week"].value_counts()
innweeks.max()

inn_id,week,target,sign,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn,cum,cumsum
str,i8,f32,i8,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,f32
"""inn999886""",105,21.231865,1,4591,1.6630e9,7,2610,21.231865,254,255,254,26990,1.9478e9,7,7612,21.38998,254,255,254,1.6570e9,6.8014e10


In [18]:
cols = innweeks.columns[4:-1]
cols

['out_vtb_trns_count',
 'out_vtb_trns_amount',
 'out_vtb_date',
 'out_vtb_cinn',
 'out_vtb_trns_amount_log',
 'out_vtbip_trns_count',
 'out_vtbip_trns_amount',
 'out_vtbip_cinn',
 'in_vtb_trns_count',
 'in_vtb_trns_amount',
 'in_vtb_date',
 'in_vtb_cinn',
 'in_vtb_trns_amount_log',
 'in_vtbip_trns_count',
 'in_vtbip_trns_amount',
 'in_vtbip_cinn',
 'cum']

In [19]:
%%time
innweeks = innweeks.with_columns(
                      #pl.col("week").rolling_min(window_size=53,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("lengthweek"),
    
                      #pl.col("sign").shift(1).over("inn_id").alias("sign1"),
                      #pl.col("sign").shift(2).over("inn_id").alias("sign2"),
                      #pl.col("sign").shift(3).over("inn_id").alias("sign3"),
                      #pl.col("sign").shift(4).over("inn_id").alias("sign4"),
                      #pl.col("sign").shift(5).over("inn_id").alias("sign5"),
                      #pl.col("sign").shift(6).over("inn_id").alias("sign6"),
                      #pl.col("sign").shift(7).over("inn_id").alias("sign7"),
                      #pl.col("sign").shift(8).over("inn_id").alias("sign8"),
                      #pl.col("sign").shift(9).over("inn_id").alias("sign9"),
                      #pl.col("sign").shift(10).over("inn_id").alias("sign10"),
                      #pl.col("sign").shift(11).over("inn_id").alias("sign11"),
                      #pl.col("sign").shift(12).over("inn_id").alias("sign12"),
                      #pl.col("sign").shift(13).over("inn_id").alias("sign13"),
                      #pl.col("sign").shift(14).over("inn_id").alias("sign14"),
                      #pl.col("sign").shift(15).over("inn_id").alias("sign15"),
                      #pl.col("sign").shift(16).over("inn_id").alias("sign16"),
                      #pl.col("sign").shift(17).over("inn_id").alias("sign17"),
                      #pl.col("sign").shift(18).over("inn_id").alias("sign18"),
                      #pl.col("sign").shift(19).over("inn_id").alias("sign19"),
    
                      #(pl.col("sign").cum_sum()/(pl.col("week")+1)).shift(1).over("inn_id").alias("active"),
                      #(pl.col("out_vtb_trns_amount").cum_sum()/pl.max_horizontal(pl.col("sign").cum_sum(),1)
                      #).shift(1).over("inn_id").alias("med"),
    
                      #pl.col("sign").rolling_sum(window_size=53,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active53"),
                      #pl.col("sign").rolling_sum(window_size=39,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active39"),
                      #pl.col("sign").rolling_sum(window_size=26,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active26"),
                      #pl.col("sign").rolling_sum(window_size=16,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active16"),
                      #pl.col("sign").rolling_sum(window_size=15,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active15"),
                      #pl.col("sign").rolling_sum(window_size=14,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active14"),
                      #pl.col("sign").rolling_sum(window_size=13,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active13"),
                      #pl.col("sign").rolling_sum(window_size=12,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active12"),
                      #pl.col("sign").rolling_sum(window_size=11,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active11"),
                      #pl.col("sign").rolling_sum(window_size=10,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active10"),
                      #pl.col("sign").rolling_sum(window_size=9,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active9"),
                      #pl.col("sign").rolling_sum(window_size=8,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active8"),
                      #pl.col("sign").rolling_sum(window_size=7,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active7"),
                      #pl.col("sign").rolling_sum(window_size=6,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active6"),
                      #pl.col("sign").rolling_sum(window_size=5,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active5"),
                      #pl.col("sign").rolling_sum(window_size=4,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active4"),
                      #pl.col("sign").rolling_sum(window_size=3,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active3"),
                      #pl.col("sign").rolling_sum(window_size=2,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active2"),
                      #pl.col("sign").rolling_sum(window_size=1,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active1"),
                      pl.col("target").shift(-1).over("inn_id").alias("target_1"),
                      pl.col("target").shift(-2).over("inn_id").alias("target_2"),
                      pl.col("target").shift(-3).over("inn_id").alias("target_3"),
    
                      cs.by_name(cols).shift(1).over("inn_id"),#.name.suffix("_01"),
                      cs.by_name(cols).shift(2).over("inn_id").name.suffix("_02"),
                      cs.by_name(cols).shift(3).over("inn_id").name.suffix("_03"),
                      cs.by_name(cols).shift(4).over("inn_id").name.suffix("_04"),
                      cs.by_name(cols).shift(5).over("inn_id").name.suffix("_05"),
                      cs.by_name(cols).shift(6).over("inn_id").name.suffix("_06"),
                      cs.by_name(cols).shift(7).over("inn_id").name.suffix("_07"),
                      cs.by_name(cols).shift(8).over("inn_id").name.suffix("_08"),
                      cs.by_name(cols).shift(9).over("inn_id").name.suffix("_09"),
                      cs.by_name(cols).shift(10).over("inn_id").name.suffix("_10"),
                      cs.by_name(cols).shift(11).over("inn_id").name.suffix("_11"),
                      cs.by_name(cols).shift(12).over("inn_id").name.suffix("_12"),
                      cs.by_name(cols).shift(13).over("inn_id").name.suffix("_13"),
                      cs.by_name(cols).shift(14).over("inn_id").name.suffix("_14"),
                      cs.by_name(cols).shift(15).over("inn_id").name.suffix("_15"),
                      cs.by_name(cols).shift(16).over("inn_id").name.suffix("_16"),
                      #cs.by_name(cols).shift(17).over("inn_id").name.suffix("_17"),
                      #cs.by_name(cols).shift(18).over("inn_id").name.suffix("_18"),
                      #cs.by_name(cols).shift(19).over("inn_id").name.suffix("_19"),
                      #cs.by_name(cols).shift(20).over("inn_id").name.suffix("_20"),
                      #cs.by_name(cols).shift(21).over("inn_id").name.suffix("_21"),
                      #cs.by_name(cols).shift(22).over("inn_id").name.suffix("_22"),
                      #cs.by_name(cols).shift(23).over("inn_id").name.suffix("_23"),
                      #cs.by_name(cols).shift(24).over("inn_id").name.suffix("_24"),
                      #cs.by_name(cols).shift(25).over("inn_id").name.suffix("_25"),
                      #cs.by_name(cols).shift(26).over("inn_id").name.suffix("_26"),
                      #cs.by_name(cols).shift(27).over("inn_id").name.suffix("_27"),
    
                      #cs.by_name(cols).shift(52).over("inn_id").name.suffix("_52"),
                      #cs.by_name(cols).shift(53).over("inn_id").name.suffix("_53"),
    
                      #pl.col("target_").shift(1).over("inn_id").alias("imax1_"),
                      #pl.col("target_").shift(2).over("inn_id").alias("imax2_"),
                      #pl.col("target_").shift(3).over("inn_id").alias("imax3_"),
                      #pl.col("target_").shift(4).over("inn_id").alias("imax4_"),
                      #pl.col("target_").shift(5).over("inn_id").alias("imax5_"),
                      #pl.col("target_").shift(6).over("inn_id").alias("imax6_"),
                      #pl.col("target_").shift(7).over("inn_id").alias("imax7_"),
                      #pl.col("target_").shift(8).over("inn_id").alias("imax8_"),
                      #pl.col("target_").shift(9).over("inn_id").alias("imax9_"),
                      #pl.col("target_").shift(10).over("inn_id").alias("imax10_"),
                      #pl.col("target_").shift(11).over("inn_id").alias("imax11_"),
                      #pl.col("target_").shift(12).over("inn_id").alias("imax12_"),
                      #pl.col("target_").shift(13).over("inn_id").alias("imax13_"),
                      #pl.col("target_").shift(14).over("inn_id").alias("imax14_"),
                      #pl.col("target_").shift(15).over("inn_id").alias("imax15_"),
                      #pl.col("target_").shift(16).over("inn_id").alias("imax16_"),
                      #pl.col("target_").shift(17).over("inn_id").alias("imax17_"),
                      #pl.col("target_").shift(18).over("inn_id").alias("imax18_"),
                      #pl.col("target_").shift(19).over("inn_id").alias("imax19_"),
                      #pl.col("target_").shift(20).over("inn_id").alias("imax20_"),
                      #pl.col("target_").shift(21).over("inn_id").alias("imax21_"),
                      #pl.col("target_").shift(22).over("inn_id").alias("imax22_"),
                      #pl.col("target_").shift(23).over("inn_id").alias("imax23_"),
                      #pl.col("target_").shift(24).over("inn_id").alias("imax24_"),
                      #pl.col("target_").shift(25).over("inn_id").alias("imax25_"),
                      #pl.col("target_").shift(26).over("inn_id").alias("imax26_"),
                      #pl.col("target_").shift(27).over("inn_id").alias("imax27_"),
                      #pl.col("target_").shift(28).over("inn_id").alias("imax28_"),
                      #pl.col("target_").shift(29).over("inn_id").alias("imax29_"),
                      #pl.col("target_").shift(30).over("inn_id").alias("imax30_"),
                      #pl.col("target_").shift(31).over("inn_id").alias("imax31_"),
                      #pl.col("target_").shift(32).over("inn_id").alias("imax32_"),
                      #pl.col("target_").shift(33).over("inn_id").alias("imax33_"),
                      #pl.col("target_").shift(34).over("inn_id").alias("imax34_"),
                      #pl.col("target_").shift(35).over("inn_id").alias("imax35_"),
                      #pl.col("target_").shift(36).over("inn_id").alias("imax36_"),
                      #pl.col("target_").shift(37).over("inn_id").alias("imax37_"),
                      #pl.col("target_").shift(38).over("inn_id").alias("imax38_"),
                      #pl.col("target_").shift(39).over("inn_id").alias("imax39_"),
                      #pl.col("target_").shift(40).over("inn_id").alias("imax40_"),
                      #pl.col("target_").shift(41).over("inn_id").alias("imax41_"),
                      #pl.col("target_").shift(42).over("inn_id").alias("imax42_"),
                      #pl.col("target_").shift(43).over("inn_id").alias("imax43_"),
                      #pl.col("target_").shift(44).over("inn_id").alias("imax44_"),
                      #pl.col("target_").shift(45).over("inn_id").alias("imax45_"),
                      #pl.col("target_").shift(46).over("inn_id").alias("imax46_"),
                      #pl.col("target_").shift(47).over("inn_id").alias("imax47_"),
                      #pl.col("target_").shift(48).over("inn_id").alias("imax48_"),
                      #pl.col("target_").shift(49).over("inn_id").alias("imax49_"),
                      #pl.col("target_").shift(50).over("inn_id").alias("imax50_"),
                      #pl.col("target_").shift(51).over("inn_id").alias("imax51_"),
                      #pl.col("target_").shift(52).over("inn_id").alias("imax52_"),
                      #pl.col("target_").shift(53).over("inn_id").alias("imax53_"),
                      #pl.col("target_").shift(54).over("inn_id").alias("imax54_"),
    
                      #pl.col("target").shift(1).over("inn_id").alias("imax1"),
                      #pl.col("target").shift(2).over("inn_id").alias("imax2"),
                      #pl.col("target").shift(3).over("inn_id").alias("imax3"),
                      #pl.col("target").shift(4).over("inn_id").alias("imax4"),
                      #pl.col("target").shift(5).over("inn_id").alias("imax5"),
                      #pl.col("target").shift(6).over("inn_id").alias("imax6"),
                      #pl.col("target").shift(7).over("inn_id").alias("imax7"),
                      #pl.col("target").shift(8).over("inn_id").alias("imax8"),
                      #pl.col("target").shift(9).over("inn_id").alias("imax9"),
                      #pl.col("target").shift(10).over("inn_id").alias("imax10"),
                      #pl.col("target").shift(11).over("inn_id").alias("imax11"),
                      #pl.col("target").shift(12).over("inn_id").alias("imax12"),
                      #pl.col("target").shift(13).over("inn_id").alias("imax13"),
                      #pl.col("target").shift(14).over("inn_id").alias("imax14"),
                      #pl.col("target").shift(15).over("inn_id").alias("imax15"),
                      #pl.col("target").shift(16).over("inn_id").alias("imax16"),
                      #pl.col("target").shift(17).over("inn_id").alias("imax17"),
                      #pl.col("target").shift(18).over("inn_id").alias("imax18"),
                      #pl.col("target").shift(19).over("inn_id").alias("imax19"),
                      #pl.col("target").shift(20).over("inn_id").alias("imax20"),
                      #pl.col("target").shift(21).over("inn_id").alias("imax21"),
                      #pl.col("target").shift(22).over("inn_id").alias("imax22"),
                      #pl.col("target").shift(23).over("inn_id").alias("imax23"),
                      #pl.col("target").shift(24).over("inn_id").alias("imax24"),
                      #pl.col("target").shift(25).over("inn_id").alias("imax25"),
                      #pl.col("target").shift(26).over("inn_id").alias("imax26"),
                      #pl.col("target").shift(27).over("inn_id").alias("imax27"),
                      #pl.col("target").shift(28).over("inn_id").alias("imax28"),
                      #pl.col("target").shift(29).over("inn_id").alias("imax29"),
                      #pl.col("target").shift(30).over("inn_id").alias("imax30"),
                      #pl.col("target").shift(31).over("inn_id").alias("imax31"),
                      #pl.col("target").shift(32).over("inn_id").alias("imax32"),
                      #pl.col("target").shift(33).over("inn_id").alias("imax33"),
                      #pl.col("target").shift(34).over("inn_id").alias("imax34"),
                      #pl.col("target").shift(35).over("inn_id").alias("imax35"),
                      #pl.col("target").shift(36).over("inn_id").alias("imax36"),
                      #pl.col("target").shift(37).over("inn_id").alias("imax37"),
                      #pl.col("target").shift(38).over("inn_id").alias("imax38"),
                      #pl.col("target").shift(39).over("inn_id").alias("imax39"),
                      #pl.col("target").shift(40).over("inn_id").alias("imax40"),
                      #pl.col("target").shift(41).over("inn_id").alias("imax41"),
                      #pl.col("target").shift(42).over("inn_id").alias("imax42"),
                      #pl.col("target").shift(43).over("inn_id").alias("imax43"),
                      #pl.col("target").shift(44).over("inn_id").alias("imax44"),
                      #pl.col("target").shift(45).over("inn_id").alias("imax45"),
                      #pl.col("target").shift(46).over("inn_id").alias("imax46"),
                      #pl.col("target").shift(47).over("inn_id").alias("imax47"),
                      #pl.col("target").shift(48).over("inn_id").alias("imax48"),
                      #pl.col("target").shift(49).over("inn_id").alias("imax49"),
                      #pl.col("target").shift(50).over("inn_id").alias("imax50"),
                      #pl.col("target").shift(51).over("inn_id").alias("imax51"),
                      #pl.col("target").shift(52).over("inn_id").alias("imax52"),
                      #pl.col("target").shift(53).over("inn_id").alias("imax53"),
                      #pl.col("target").shift(54).over("inn_id").alias("imax54"),
    
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      #pl.col("target").rolling_max(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imax"),
                      #pl.col("target").rolling_min(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imin"),
                      #pl.col("target_").rolling_mean(window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("imean"),
                      #pl.col("target_").rolling_median(window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian"),
    
                      #pl.col("target_").rolling_mean(window_size=2,min_periods=1).shift(1).over("inn_id").log1p().alias("imean2"),
                      #pl.col("target_").rolling_median(window_size=2,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian2"),
    
                      #pl.col("target_").rolling_mean(window_size=3,min_periods=1).shift(1).over("inn_id").log1p().alias("imean3"),
                      #pl.col("target_").rolling_median(window_size=3,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian3"),
    
                      #pl.col("target_").rolling_mean(window_size=4,min_periods=1).shift(1).over("inn_id").log1p().alias("imean4"),
                      #pl.col("target_").rolling_median(window_size=4,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian4"),
    
                      #pl.col("target_").rolling_mean(window_size=5,min_periods=1).shift(1).over("inn_id").log1p().alias("imean5"),
                      #pl.col("target_").rolling_median(window_size=5,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian5"),
    
                      #pl.col("target_").rolling_mean(window_size=13,min_periods=1).shift(1).over("inn_id").log1p().alias("imean13"),
                      #pl.col("target_").rolling_median(window_size=13,min_periods=1).shift(1).over("inn_id").log1p().alias("imedian13"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq095"),
                      #pl.col("target_").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq090"),
                      #pl.col("target_").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq085"),
                      #pl.col("target_").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq080"),
                      #pl.col("target_").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq075"),
                      #pl.col("target_").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq070"),
                      #pl.col("target_").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq065"),
                      #pl.col("target_").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq060"),
                      #pl.col("target_").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq055"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq045"),
                      #pl.col("target_").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq040"),
                      #pl.col("target_").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq035"),
                      #pl.col("target_").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq030"),
                      #pl.col("target_").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq025"),
                      #pl.col("target_").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq020"),
                      #pl.col("target_").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq015"),
                      #pl.col("target_").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq010"),
                      #pl.col("target_").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(1).over("inn_id").log1p().alias("iq005"),
    
                      #pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(1).over("inn_id").alias("mean53"),
                      #pl.col("target").rolling_mean(window_size=13,min_periods=1).shift(1).over("inn_id").alias("mean13"),
                      #pl.col("target").rolling_median(window_size=53,min_periods=1).shift(1).over("inn_id").alias("median53"),
                      #pl.col("target").rolling_median(window_size=13,min_periods=1).shift(1).over("inn_id").alias("median13"),
                      #pl.col("out_vtb_trns_amount").rolling_mean(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imean_"),
                      #pl.col("out_vtb_trns_amount").rolling_median(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imedian_"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq095_"),
                      #pl.col("target_").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq090_"),
                      #pl.col("target_").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq085_"),
                      #pl.col("target_").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq080_"),
                      #pl.col("target_").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq075_"),
                      #pl.col("target_").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq070_"),
                      #pl.col("target_").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq065_"),
                      #pl.col("target_").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq060_"),
                      #pl.col("target_").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq055_"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq045_"),
                      #pl.col("target_").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq040_"),
                      #pl.col("target_").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq035_"),
                      #pl.col("target_").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq030_"),
                      #pl.col("target_").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq025_"),
                      #pl.col("target_").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq020_"),
                      #pl.col("target_").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq015_"),
                      #pl.col("target_").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq010_"),
                      #pl.col("target_").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq005_"),
                     )
innweeks

CPU times: user 35.1 s, sys: 9.38 s, total: 44.5 s
Wall time: 41.8 s


inn_id,week,target,sign,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn,cum,cumsum,target_1,target_2,target_3,out_vtb_trns_count_02,out_vtb_trns_amount_02,out_vtb_date_02,out_vtb_cinn_02,out_vtb_trns_amount_log_02,out_vtbip_trns_count_02,out_vtbip_trns_amount_02,out_vtbip_cinn_02,in_vtb_trns_count_02,in_vtb_trns_amount_02,in_vtb_date_02,in_vtb_cinn_02,…,in_vtbip_trns_amount_14,in_vtbip_cinn_14,cum_14,out_vtb_trns_count_15,out_vtb_trns_amount_15,out_vtb_date_15,out_vtb_cinn_15,out_vtb_trns_amount_log_15,out_vtbip_trns_count_15,out_vtbip_trns_amount_15,out_vtbip_cinn_15,in_vtb_trns_count_15,in_vtb_trns_amount_15,in_vtb_date_15,in_vtb_cinn_15,in_vtb_trns_amount_log_15,in_vtbip_trns_count_15,in_vtbip_trns_amount_15,in_vtbip_cinn_15,cum_15,out_vtb_trns_count_16,out_vtb_trns_amount_16,out_vtb_date_16,out_vtb_cinn_16,out_vtb_trns_amount_log_16,out_vtbip_trns_count_16,out_vtbip_trns_amount_16,out_vtbip_cinn_16,in_vtb_trns_count_16,in_vtb_trns_amount_16,in_vtb_date_16,in_vtb_cinn_16,in_vtb_trns_amount_log_16,in_vtbip_trns_count_16,in_vtbip_trns_amount_16,in_vtbip_cinn_16,cum_16
str,i8,f32,i8,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,f32,f32,f32,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,…,u8,u8,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32
"""inn1000051""",0,12.95314,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,7.392578,13.929488,10.583951,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",1,7.392578,0,2,422159.3125,2,2,12.95314,0,0,0,5,36499.808594,3,1,10.50509,0,0,0,-385659.53125,13733.0625,13.929488,10.583951,12.972155,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",2,13.929488,0,1,1622.887451,1,1,7.392578,0,0,0,9,643022.5625,3,5,13.373937,56,130,101,641399.6875,655132.75,10.583951,12.972155,0.0,2,422159.3125,2,2,12.95314,0,0,0,5,36499.808594,3,1,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",3,10.583951,0,1,1.1207e6,1,1,13.929488,0,0,0,9,433354.75,4,3,12.979314,113,238,169,-687370.875,13069.5,12.972155,0.0,13.739822,1,1622.887451,1,1,7.392578,0,0,0,9,643022.5625,3,5,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",4,12.972155,0,1,39494.851562,1,1,10.583951,0,0,0,5,10727.206055,4,1,9.280632,0,0,0,-28767.646484,0.0,0.0,13.739822,14.26319,1,1.1207e6,1,1,13.929488,0,0,0,9,433354.75,4,3,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999886""",101,15.637834,0,72,1.794848e7,6,31,16.703016,38,1,49,59,2.4776296e7,6,40,17.025398,21,4,25,6827815.5,8.78596928e8,17.042799,16.736929,17.025045,74,2.0867246e7,6,23,16.853691,6,0,22,39,2.0877604e7,5,27,…,45,34,1.694349e7,47,9.711682e6,6,28,16.08884,27,2,36,49,2.3663876e7,5,28,16.97946,15,8,18,1.3952195e7,6

In [20]:
calendar = pl.read_csv("data/calendar.csv", try_parse_dates=True
                      ).with_columns(
                              pl.col("week").cast(pl.Int8),
)
calendar

date,week,part
date,i8,str
2022-07-25,0,"""train"""
2022-07-26,0,"""train"""
2022-07-27,0,"""train"""
2022-07-28,0,"""train"""
2022-07-29,0,"""train"""
…,…,…
2024-10-23,117,"""private"""
2024-10-24,117,"""private"""
2024-10-25,117,"""private"""


In [21]:
weeks = calendar.group_by("week").agg(
                              pl.col("date").min().alias("d1"),
                              pl.col("date").max().alias("d2"),
                             ).with_columns(
                              pl.col("week").cast(pl.Int8),
    
                              pl.col("d1").dt.year().cast(pl.Int16).alias("year1"),
                              pl.col("d1").dt.month().alias("month1"),
                              pl.col("d1").dt.day().alias("day1"),
                              pl.col("d1").dt.ordinal_day().alias("dy1"),
    
                              pl.col("d2").dt.year().cast(pl.Int16).alias("year2"),
                              pl.col("d2").dt.month().alias("month2"),
                              pl.col("d2").dt.day().alias("day2"),
                              pl.col("d2").dt.ordinal_day().alias("dy2"),
                             ).with_columns(
                              ((pl.col("month2")-pl.col("month1")+12)%12).alias("m"),
                              (pl.col("year2")-pl.col("year1")).cast(pl.Int8).alias("y"),
    
                              pl.when( pl.col("month2").is_in([1,3,5,7,8,10,12])
                                     ).then(31-pl.col("day2")
                                           ).otherwise(
                              pl.when( pl.col("month2").is_in([2,4,6,9,11])
                                     ).then(30-pl.col("day2")
                                           ).otherwise(
                                                       28-pl.col("day2")+(pl.col("year2")==2024)
                                                      ),
                                                       
                                                      ).alias("day2_"),
    
                              pl.when( pl.col("month1").is_in([1,3,5,7,8,10,12])
                                     ).then(31-pl.col("day1")
                                           ).otherwise(
                              pl.when( pl.col("month1").is_in([2,4,6,9,11])
                                     ).then(30-pl.col("day1")
                                           ).otherwise(
                                                       28-pl.col("day1")+(pl.col("year1")==2024)
                                                      ),
                                                       
                                                      ).alias("day1_"),
                                
    
                              pl.when( (pl.col("month1")>2)&(pl.col("year1")%4!=0)).then(pl.col("dy1")+1).otherwise(pl.col("dy1")).alias("dy1"),
                              pl.when( (pl.col("month2")>2)&(pl.col("year2")%4!=0)).then(pl.col("dy2")+1).otherwise(pl.col("dy2")).alias("dy2"),
                              #(pl.col("d1")+datetime.timedelta(-1)).replace(workdays).cast(pl.Int8).alias("work_1"), 
                              (
                              (pl.col("d1")+datetime.timedelta(0)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(1)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(2)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(3)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(4)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(5)).replace(workdays).cast(pl.Int8)+
                              (pl.col("d1")+datetime.timedelta(6)).replace(workdays).cast(pl.Int8)
                              ).alias("w7"), 
                              #(pl.col("d1")+datetime.timedelta(7)).replace(workdays).cast(pl.Int8).alias("work8"), 
                             #).with_columns( 
                             # (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")+pl.col("work5")+pl.col("work6")+pl.col("work7")).alias("w7"), 
                             ).with_columns( 
                              pl.col("w7").shift(-3).alias("w7__3"), 
                              pl.col("w7").shift(-2).alias("w7__2"), 
                              pl.col("w7").shift(-1).alias("w7__1"), 
    
                              #pl.col("w7").shift(1).alias("w7_1"), 
                              #pl.col("w7").shift(2).alias("w7_2"), 
                              #pl.col("w7").shift(3).alias("w7_3"), 
                              #pl.col("w7").shift(4).alias("w7_4"), 
                              #pl.col("w7").shift(5).alias("w7_5"), 
                              #pl.col("w7").shift(6).alias("w7_6"), 
                             )
weeks=weeks.sort("week")
weeks

week,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y,day2_,day1_,w7,w7__3,w7__2,w7__1
i8,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8,i8,i8,i8,i8,i8,i8
0,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4
1,2022-08-01,2022-08-07,2022,8,1,214,2022,8,7,220,0,0,24,30,5,3,5,5
2,2022-08-08,2022-08-14,2022,8,8,221,2022,8,14,227,0,0,17,23,5,4,5,5
3,2022-08-15,2022-08-21,2022,8,15,228,2022,8,21,234,0,0,10,16,5,5,5,5
4,2022-08-22,2022-08-28,2022,8,22,235,2022,8,28,241,0,0,3,9,5,5,5,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
113,2024-09-23,2024-09-29,2024,9,23,267,2024,9,29,273,0,0,1,7,5,5,5,5
114,2024-09-30,2024-10-06,2024,9,30,274,2024,10,6,280,1,0,25,0,5,null,null,null
115,2024-10-07,2024-10-13,2024,10,7,281,2024,10,13,287,0,0,18,24,5,5,5,5


In [22]:
wcols=weeks.columns[3:]
wcols

['year1',
 'month1',
 'day1',
 'dy1',
 'year2',
 'month2',
 'day2',
 'dy2',
 'm',
 'y',
 'day2_',
 'day1_',
 'w7',
 'w7__3',
 'w7__2',
 'w7__1']

In [23]:
#weeks=weeks.with_columns(#cs.by_name(wcols).shift(-1).name.suffix("_02"),
#                         cs.by_name(wcols).shift(-2).name.suffix("_03"),
#                         #cs.by_name(wcols).shift(-3).name.suffix("_04"),
#                        )
#week = weeks.sort("week")
#weeks

In [24]:
#profiles = pl.read_parquet("data/profiles.parquet")#[""]
#profiles#["ipul"].unique()

#okved = pl.read_parquet("data/profiles.parquet"
#               ).pivot(index="inn_id", on="main_okved_group", values="ipul" , aggregate_function="count"
#                      ).fill_null(0).rename(lambda column_name: "okved_" + column_name
#                                           ).with_columns(pl.selectors.integer().cast(pl.UInt8), 
#                                                         )
#okved

In [25]:
#region=pl.read_parquet("data/profiles.parquet"
#               ).pivot(index="inn_id", on="id_region", values="ipul" , aggregate_function="count"
#                      ).fill_null(0).rename(lambda column_name: "region_" + column_name
#                                           ).with_columns(pl.selectors.integer().cast(pl.UInt8), 
#                                                         )
#region

In [26]:
profiles = pl.read_parquet("data/profiles.parquet")#["diff_datopen_report_date_flg"].unique()
age={
    "1m":1,
    "2_3m":3,
    "3_6m":6,
    "6_12m":12,
    "1_2y":24,
    "2_4y":48,
    "4_8y":96,
    "8_12y":144,
    "more_12y":192,
    #"NULL":np.nan
}    
ipul={
    "ip":0,
    "ul":1,
}    
profiles = profiles.with_columns(
    pl.col("report_date").str.to_date(),
    pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),
    pl.col("ipul").replace(ipul).cast(pl.Int8),
    pl.col("id_region").fill_null("-"),
    pl.col("main_okved_group").fill_null("-"),
    pl.col("inn_id").str.replace("inn","").cast(pl.Int32).alias("inn"),
)
profiles = profiles.sort("report_date"
                        ).group_by("inn_id"
                                  ).agg(pl.col("ipul").first(),
                                        pl.col("id_region").first(),
                                        pl.col("main_okved_group").last(),
                                        pl.col("diff_datopen_report_date_flg").max(),
                                        pl.col("diff_datopen_report_date_flg").min().alias("diff_datopen_report_date_flg_min"),
                                        
                                        #pl.col("report_date").max(),
                                        #pl.col("report_date").min().alias("report_date_min"),
                                        #pl.col("id_region").unique().alias("id_region"),
                                        #pl.col("main_okved_group").unique().alias("main_okved_group"),
                                        #pl.col("diff_datopen_report_date_flg").n_unique().alias("diff_datopen_report_date_count"),
                                       ).join(profiles.group_by("inn_id"
                                                               ).agg(
                                        #pl.col("report_date").max(),
                                        pl.col("report_date").min().alias("report_date_min"),
                                                                     ),
                                              on="inn_id",
                                              how="left"
                                        #).with_columns(
                                        #    pl.col("inn_id").str.replace("inn","").cast(pl.UInt32).alias("inn"),
                                        )
profiles
#profiles=profiles.join(okved,how="left",left_on="inn_id",right_on="okved_inn_id"
#                      ).join(region,how="left",left_on="inn_id",right_on="region_inn_id"
#                            )
profiles

/tmp/ipykernel_75170/193819381.py:20: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),


inn_id,ipul,id_region,main_okved_group,diff_datopen_report_date_flg,diff_datopen_report_date_flg_min,report_date_min
str,i8,str,str,f64,f64,date
"""inn692409""",0,"""46""","""46""",192.0,192.0,2023-10-01
"""inn1559860""",1,"""40""","""62""",144.0,144.0,2023-10-01
"""inn1932166""",0,"""46""","""14""",144.0,144.0,2023-10-01
"""inn506801""",1,"""89""","""01""",192.0,192.0,2023-10-01
"""inn1148796""",1,"""36""","""49""",24.0,6.0,2023-10-01
…,…,…,…,…,…,…
"""inn1439769""",0,"""41""","""41""",48.0,24.0,2023-10-01
"""inn1245885""",1,"""92""","""90""",96.0,96.0,2023-10-01
"""inn2333089""",1,"""46""","""77""",48.0,48.0,2023-10-01


In [27]:
#profiles2 = profiles.join(innstat0,on="inn_id",how="left")
#profiles.max()

#weekclass = pl.read_parquet("weekclass.parquet")
#weekclass

In [28]:
XX = innweeks.join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        ).sort("week"
                 # ).join(weekclass.with_columns(pl.col("class_nweek")+1),
                 #        left_on=["inn_id","week"],
                 #        right_on=["class_doc_payer_inn","class_nweek"],
                 #        how="left"
                 # ).join(weekclass.with_columns(pl.col("class_nweek")+2),
                 #        left_on=["inn_id","week"],
                 #        right_on=["class_doc_payer_inn","class_nweek"],
                 #        suffix="02",
                 #        how="left"
                 # ).join(weekclass.with_columns(pl.col("class_nweek")+3),
                 #        left_on=["inn_id","week"],
                 #        right_on=["class_doc_payer_inn","class_nweek"],
                 #        suffix="03",
                 #        how="left"
                 # ).join(weekclass.with_columns(pl.col("class_nweek")+4),
                 #        left_on=["inn_id","week"],
                 #        right_on=["class_doc_payer_inn","class_nweek"],
                 #        suffix="04",
                 #        how="left"
                                #.with_columns(pl.col("").cast(pl.String),
                                #              pl.col("").cast(pl.String),
                                              )
#.filter(pl.col("week")>4)
XX

inn_id,week,target,sign,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn,cum,cumsum,target_1,target_2,target_3,out_vtb_trns_count_02,out_vtb_trns_amount_02,out_vtb_date_02,out_vtb_cinn_02,out_vtb_trns_amount_log_02,out_vtbip_trns_count_02,out_vtbip_trns_amount_02,out_vtbip_cinn_02,in_vtb_trns_count_02,in_vtb_trns_amount_02,in_vtb_date_02,in_vtb_cinn_02,…,out_vtb_trns_amount_log_16,out_vtbip_trns_count_16,out_vtbip_trns_amount_16,out_vtbip_cinn_16,in_vtb_trns_count_16,in_vtb_trns_amount_16,in_vtb_date_16,in_vtb_cinn_16,in_vtb_trns_amount_log_16,in_vtbip_trns_count_16,in_vtbip_trns_amount_16,in_vtbip_cinn_16,cum_16,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y,day2_,day1_,w7,w7__3,w7__2,w7__1,ipul,id_region,main_okved_group,diff_datopen_report_date_flg,diff_datopen_report_date_flg_min,report_date_min
str,i8,f32,i8,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,f32,f32,f32,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,…,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8,i8,i8,i8,i8,i8,i8,i8,str,str,f64,f64,date
"""inn1000051""",0,12.95314,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,7.392578,13.929488,10.583951,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4,0,"""45""","""68""",96.0,48.0,2023-10-01
"""inn1000246""",0,13.911296,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,13.679488,13.335955,12.630267,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4,1,"""33""","""17""",144.0,96.0,2023-10-01
"""inn1000281""",0,0.0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,10.467411,11.994005,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4,1,"""40""","""20""",192.0,192.0,2023-10-01
"""inn1000367""",0,12.620393,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,8.08754,12.917565,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4,1,"""41""","""25""",96.0,96.0,2023-10-01
"""inn1000409""",0,12.576319,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,11.939093,11.059709,10.855669,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,0,6,5,5,4,4,0,"""45""","""68""",192.0,192.0,2023-10-01
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999585""",105,15.368829,0,23,6.651852e6,5,16,15.710406,88,20,95,11,1.9563e6,4,4,14.486568,92,68,191,-4.6955e6,0.0,null,null,null,20,4845396.5,5,10,15.393539,25,0,50,19,3.6557e6,5,3,…,15.510999,22,27,39,13,2.039293e6,4,3,14.528114,0,0,0,-3.410011e6,2024-07-29,2024-08-04,2024,7,29,211,2024,8,4,217,1,0,27,2,5,5,5,5,1,"""70""","""35""",192.0,192.0,2023-10-01
"""inn999624""",105,12.198977,0,9,327611.9375,2,9,12.699588,0,0,0,0,0.0,0,0,0.0,0,0,0,-327611.9375,0.0,null,null,null,15,416181.9375,3,9,12.93888,0,0,0,0,0.0,0,0,…,12.133861,0,0,0,0,0.0,0,0,0.0,0,0,0,-186065.75,2024-07-29,2024-08-04,2024,7,29,211,2024,8,4,217,1,0

In [29]:
XX.columns

['inn_id',
 'week',
 'target',
 'sign',
 'out_vtb_trns_count',
 'out_vtb_trns_amount',
 'out_vtb_date',
 'out_vtb_cinn',
 'out_vtb_trns_amount_log',
 'out_vtbip_trns_count',
 'out_vtbip_trns_amount',
 'out_vtbip_cinn',
 'in_vtb_trns_count',
 'in_vtb_trns_amount',
 'in_vtb_date',
 'in_vtb_cinn',
 'in_vtb_trns_amount_log',
 'in_vtbip_trns_count',
 'in_vtbip_trns_amount',
 'in_vtbip_cinn',
 'cum',
 'cumsum',
 'target_1',
 'target_2',
 'target_3',
 'out_vtb_trns_count_02',
 'out_vtb_trns_amount_02',
 'out_vtb_date_02',
 'out_vtb_cinn_02',
 'out_vtb_trns_amount_log_02',
 'out_vtbip_trns_count_02',
 'out_vtbip_trns_amount_02',
 'out_vtbip_cinn_02',
 'in_vtb_trns_count_02',
 'in_vtb_trns_amount_02',
 'in_vtb_date_02',
 'in_vtb_cinn_02',
 'in_vtb_trns_amount_log_02',
 'in_vtbip_trns_count_02',
 'in_vtbip_trns_amount_02',
 'in_vtbip_cinn_02',
 'cum_02',
 'out_vtb_trns_count_03',
 'out_vtb_trns_amount_03',
 'out_vtb_date_03',
 'out_vtb_cinn_03',
 'out_vtb_trns_amount_log_03',
 'out_vtbip_trns_co

In [30]:
y = XX.select("target",
              "target_1",
              "target_2","target_3"
             ).to_numpy()
X = XX.drop(["target","target_1","target_2","target_3",
             #"target_",
             "sign",
             "d1",
             "d2",
             #"week",
             #"inn_id",
             #'inn_id_02',
             #'inn_id_03',
             #'inn_id_04',
             #"report_date"
            ]).to_pandas()
#X_train, X_test, y_train, y_test = train_test_split(    X, y, test_size=0.01, random_state=42)
#border=102
border=delweek
X_train = X[X["week"]<border]#.drop("week",axis=1) 
y_train = y[X["week"]<border]    

#train_data.quantize()
#train_data = Pool(data=X[X["week"]<border],#.drop("week",axis=1) 
#                  label=y[X["week"]<border],
#                  cat_features=["inn_id",
#                                          "ipul",
#                                          "id_region",
#                                          "main_okved_group",
#                                          #"report_date"
#                                         ],
#                 )
#train_data.quantize()
#train_data.save("train_data.pool")

#X_train = X.drop("week",axis=1) 
#y_train = y

X_test = X[X["week"].isin(range(border,103))]#.drop("week",axis=1) 
y_test = y[X["week"].isin(range(border,103))] 

#test_data = Pool(data=X[X["week"].isin(range(border,103))],#.drop("week",axis=1) 
#                 label=y[X["week"].isin(range(border,103))],
#                 cat_features=["inn_id","ipul","id_region","main_okved_group",
#                                          #"report_date"
#                                         ],
#                )
#test_data.quantize()
#test_data.save("test_data.pool")

#X_test = X[X["week"]==border].drop("week",axis=1) 
#y_test = y[X["week"]==border] 
X_test

,inn_id,week,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,...,w7,w7__3,w7__2,w7__1,ipul,id_region,main_okved_group,diff_datopen_report_date_flg,diff_datopen_report_date_flg_min,report_date_min
5300226,inn1000051,102,3.0,6.819989e+04,2.0,2.0,11.130213,0.0,0.0,0.0,...,5,5.0,5.0,5,0,45,68,96.0,48.0,2023-10-01
5300227,inn1000246,102,33.0,5.521449e+06,5.0,20.0,15.524151,15.0,14.0,25.0,...,5,5.0,5.0,5,1,33,17,144.0,96.0,2023-10-01
5300228,inn1000281,102,5.0,1.842286e+05,2.0,5.0,12.123939,50.0,0.0,50.0,...,5,5.0,5.0,5,1,40,20,192.0,192.0,2023-10-01
5300229,inn1000367,102,13.0,6.240296e+05,3.0,9.0,13.343955,0.0,0.0,0.0,...,5,5.0,5.0,5,1,41,25,96.0,96.0,2023-10-01
5300230,inn1000409,102,1.0,1.996111e+03,1.0,1.0,7.599457,0.0,0.0,0.0,...,5,5.0,5.0,5,0,45,68,192.0,192.0,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352184,inn999585,102,27.0,8.546690e+06,5.0,19.0,15.961055,28.0,29.0,40.0,...,5,5.0,5.0,5,1,70,35,192.0,192.0,2023-10-01
5352185,inn999624,102,14.0,2.749947e+05,4.0,8.0,12.524510,0.0,0.0,0.0,...,5,5.0,5.0,5,0,46,47,192.0,192.0,2023-10-01
5352186,inn999737,102,9.0,4.741556e+06,3.0,9.0,15.371876,0.0,0.0,0.0,...,5,5.0,5.0,5,1,46,22,96.0,96.0,2023-10-01
5352187,inn999759,102,61.0,7.971220e+06,7.0,47.0,15.891348,62.0,32.0,59.0,...,5,5.0,5.0,5,1,70,25,144.0,144.0,2023-10-01


In [31]:
#del in_vtbip
#del in_vtb
#del out_vtbip
#del out_vtb
#del train_data
#del test_data
#del innweeks
#del XX
#del X, y
#gc.collect()

In [32]:
y_test

array([[15.044251, 14.610071, 11.487807, 15.821142],
       [15.053899, 15.62723 , 15.110888, 15.626517],
       [12.78503 ,  9.141981, 10.774144, 10.328082],
       ...,
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [16.11803 , 16.202227, 16.094133, 16.255236],
       [17.042799, 16.736929, 17.025045, 16.799984]], dtype=float32)

In [33]:
model = CatBoostRegressor(
#model = CatBoostClassifier(
                            loss_function='MultiRMSE', 
                            #custom_metric=["MAPE","MAE",#"MSLE"
                            #loss_function=['MultiLogloss', 
                            #loss_function='MultiCrossEntropy', 
                            #custom_metric=["Accuracy","F1","Precision","Recall"#"MSLE"
                            #              ],
                            iterations=3000,
                            #learning_rate=0.20126033,
                            #learning_rate=0.01526033, #0.156796
                            learning_rate=0.156796,
                            depth=8,
                            cat_features=["inn_id",
                                          "ipul",
                                          "id_region",
                                          "main_okved_group",
                                          #"report_date"
                                         ],
                            one_hot_max_size=256,
                            #l2_leaf_reg=1,
                            task_type="GPU",
                            model_size_reg=16,
                            #grow_policy="Depthwise",
                            #grow_policy="Lossguide",
                            random_seed=0,
                            has_time=True,
                            #save_snapshot=True,
                         )
model.fit(
          X_train,y_train,
          eval_set=(X_test, y_test),
          plot=True,
          verbose=100,
          use_best_model=True,
          early_stopping_rounds=100,
         )
#model.save_model(f"model{model.learning_rate_}_{model.best_score_}.cbm")
#bestTest = 4.351532023
#bestTest = 4.414054321
#bestTest = 4.407048532
#bestTest = 4.404049331 i1000
#bestTest = 4.40182103 bestIteration = 1253
#bestTest = 4.425927569 bestIteration = 852
#bestTest = 4.427390062 bestIteration = 923
#bestTest = 4.405004967 bestIteration = 1332
#bestTest = 4.399930719 bestIteration = 1883

#bestTest = 3.936813602 bestIteration = 53 d10 ohm256 lr0915
#bestTest = 3.891957966 bestIteration = 231 d10 ohm256 lr015
#bestTest = 3.881777939 bestIteration = 803 d10 ohm256 lr015 
#bestTest = 3.790749576 bestIteration = 1255 -out count 
#bestTest = 3.79126879 bestIteration = 938

#bestTest = 4.349578194 bestIteration = 928 d10 lr015
#bestTest = 4.329466432 bestIteration = 2991 d6 lr015
#bestTest = 4.335003995 bestIteration = 861 d10 lr015
#bestTest = 4.34124022 bestIteration = 785 d10 lr015
#bestTest = 4.332035294 bestIteration = 2998 d6

#bestTest = 4.346834727 bestIteration = 815 d10
#bestTest = 4.334658408 bestIteration = 1969 d6
#bestTest = 4.348680631 bestIteration = 2962 d4
#bestTest = 4.334532985 bestIteration = 1741 d8
#bestTest = 4.330862352 bestIteration = 2218 d7

#bestTest = 2.01308642 bestIteration = 1017 d10
#bestTest = 2.016137679 bestIteration = 2353 d6
#bestTest = 2.018460227 bestIteration = 671 d12
#bestTest = 2.010043545 bestIteration = 2185 d8
#bestTest = 2.017218533 bestIteration = 1462 d7
#bestTest = 2.012027183 bestIteration = 1055 d9
#bestTest = 2.009512633 bestIteration = 2604 d8 lr00915

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 7.3808124	test: 6.1083475	best: 6.1083475 (0)	total: 300ms	remaining: 14m 59s
100:	learn: 4.9945652	test: 4.4325485	best: 4.4325485 (100)	total: 27.3s	remaining: 13m 3s
200:	learn: 4.9610195	test: 4.4007056	best: 4.4007056 (200)	total: 54.2s	remaining: 12m 35s
300:	learn: 4.9433828	test: 4.3866205	best: 4.3866205 (300)	total: 1m 19s	remaining: 11m 57s
400:	learn: 4.9298220	test: 4.3796855	best: 4.3796855 (400)	total: 1m 45s	remaining: 11m 25s
500:	learn: 4.9192539	test: 4.3758222	best: 4.3758001 (499)	total: 2m 10s	remaining: 10m 52s
600:	learn: 4.9098369	test: 4.3740110	best: 4.3739482 (589)	total: 2m 36s	remaining: 10m 23s
700:	learn: 4.9013616	test: 4.3715040	best: 4.3715040 (700)	total: 3m 1s	remaining: 9m 53s
800:	learn: 4.8937248	test: 4.3698406	best: 4.3698406 (800)	total: 3m 26s	remaining: 9m 26s
900:	learn: 4.8860009	test: 4.3673521	best: 4.3672856 (894)	total: 3m 50s	remaining: 8m 57s
1000:	learn: 4.8787766	test: 4.3660199	best: 4.3659149 (969)	total: 4m 15s	remaini

In [34]:
"""
model2 = CatBoostRegressor(
                            #loss_function='MultiRMSE', 
                            #iterations=100,
                            #learning_rate=0.20126033,
                            #learning_rate=0.01526033, #0.156796
                            learning_rate=0.0156796,
                            depth=10,
                            cat_features=["inn_id",
                                          "ipul",
                                          "id_region",
                                          "main_okved_group",
                                          #"report_date"
                                         ],
                            one_hot_max_size=256,
                            #l2_leaf_reg=3,
                            #task_type="GPU",
                            #model_size_reg=16,
                            #grow_policy="Depthwise",
                            #grow_policy="Lossguide",
                            random_seed=0,
                            has_time=True,
                         )
model2.fit(X_train,y_train,
          eval_set=(X_test, y_test),
          plot=True,
          verbose=100,
          use_best_model=True,
          early_stopping_rounds=10,
          init_model=model
         )
"""
0

0

In [35]:
#model.fit(X_train,y_train,
#          eval_set=(X_test, y_test),
#          plot=True,
#          verbose=100,
#          use_best_model=True,
#          early_stopping_rounds=10,
#          init_model=model,
#         )

In [36]:
"""
y = XX.select("target",
              #"target_1","target_2","target_3"
             ).to_numpy()
X = XX.drop(["target","target_1","target_2","target_3",
             "sign",
             "d1",
             "d2",
             #"week",
             #"inn_id",
             #"report_date"
            ]).to_pandas()

cv_dataset = Pool(data=X[X["week"]<103],
                  label=y[X["week"]<103],
                 cat_features=["inn_id",
                               "ipul",
                               "id_region",
                               "main_okved_group",
                               #"report_date"
                               ],                  
                  #text_features=["source_name","wm"]
                 )

catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.156796,
    #'learning_rate': 0.01,
    #'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'depth':8,
    'early_stopping_rounds': 50,
    'use_best_model': True,
    'verbose': 100,
    #'boosting_type':"Plain",
    
    #"loss_function": "MultiRMSE",
    "loss_function": "RMSE",
   
}

scores = cv(cv_dataset,
            catboost_params,
            fold_count=7,
            plot=True,
            return_models=True,
            stratified=False,
            #type='Classical',
            #type='TimeSeries',
           )
scores
"""
0

0

In [37]:
#model.save_model(f"model{model.learning_rate_}_{model.best_score_}.cbm")

In [38]:
#XX["target"].is_null().sum()
#X
model.get_feature_importance(prettified=True).head(60)

,Feature Id,Importances
0,dy1,12.530624
1,out_vtb_trns_count,11.302700
2,dy2,7.177636
3,out_vtb_trns_amount_02,3.859169
4,out_vtb_trns_amount,3.659748
5,out_vtb_trns_amount_04,3.599397
6,cum_02,3.251163
7,cum,2.964985
8,day2,2.964743
9,out_vtb_cinn_03,2.819890


In [39]:
model.get_feature_importance(prettified=True)[60:120]

,Feature Id,Importances
60,out_vtb_trns_count_13,0.148904
61,out_vtb_trns_count_09,0.146351
62,out_vtb_trns_count_10,0.143043
63,out_vtb_trns_count_15,0.136536
64,cum_16,0.132728
65,in_vtb_trns_amount_log_02,0.129639
66,out_vtb_trns_amount_15,0.128919
67,in_vtb_trns_amount_02,0.127526
68,out_vtb_trns_amount_log_11,0.123697
69,out_vtb_trns_amount_log_09,0.122346


In [40]:
model.get_feature_importance(prettified=True).tail(60)

,Feature Id,Importances
237,out_vtbip_trns_amount_12,0.008983
238,in_vtbip_cinn_14,0.008929
239,in_vtb_trns_amount_log_04,0.008741
240,out_vtbip_cinn_12,0.008618
241,in_vtbip_trns_count_06,0.008578
242,in_vtb_date_10,0.008577
243,out_vtbip_cinn_08,0.008352
244,in_vtbip_trns_count_14,0.008245
245,in_vtb_trns_amount_log_03,0.008242
246,in_vtb_date_14,0.008226


In [41]:
%%time
innweeks0 = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                #pl.col("target").alias("target_"),
                            #).with_columns(                              
                                #np.log1p(pl.col("target_")).alias("target"),
                            ).with_columns(                              
                                pl.col("target").sign().cast(pl.Float32).alias("sign"),
                            #).with_columns(                              
                            #    (1-pl.col("target").sign()).cast(pl.Float32).alias("sign"),
                            ).join(out_vtb,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payer_inn","week"],
                                   how="left"
                            ).join(out_vtbip,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payer_inn","week"],
                                   how="left"
                            #).join(out_notvtb,
                            #       left_on=["inn_id","week"],
                            #       right_on=["doc_payer_inn","week"],
                            #       how="left"
                            ).join(in_vtb,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payee_inn","week"],
                                   how="left"
                           ).join(in_vtbip,
                                   left_on=["inn_id","week"],
                                   right_on=["doc_payee_inn","week"],
                                   how="left"
                            #).join(in_notvtb,
                            #       left_on=["inn_id","week"],
                            #       right_on=["doc_payee_inn","week"],
                            #       how="left"
                            ).fill_null(0
                            ).with_columns(#cs.by_dtype(pl.UInt32).cast(pl.UInt8),
                                           (pl.col("in_vtb_trns_amount")-pl.col("out_vtb_trns_amount")
                                           ).cast(pl.Float32).alias("cum"),
                                           cs.ends_with("_date").cast(pl.UInt8),
                                           cs.ends_with("_class_encoded").cast(pl.UInt8),
                                           cs.ends_with("_name_encoded").cast(pl.UInt16),
                                           cs.ends_with("_trns_count").cast(pl.UInt16),
                                           #cs.starts_with("target").cast(pl.Float32),
                                           cs.ends_with("_trns_amount").cast(pl.Float32),
                                           cs.ends_with("_log").cast(pl.Float32),
                                           #cs.by_dtype(pl.Float64).cast(pl.Float32),
                            ).with_columns(
                                           pl.col("cum").cum_sum().over("inn_id").alias("cumsum"),
                                           (pl.col("out_vtbip_trns_amount")/(pl.col("out_vtb_trns_amount")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_trns_amount")/(pl.col("in_vtb_trns_amount")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("out_vtbip_trns_count")/(pl.col("out_vtb_trns_count")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_trns_count")/(pl.col("in_vtb_trns_count")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("out_vtbip_cinn")/(pl.col("out_vtb_cinn")+0.00001)*255
                                           ).cast(pl.UInt8),
                                           (pl.col("in_vtbip_cinn")/(pl.col("in_vtb_cinn")+0.00001)*255
                                           ).cast(pl.UInt8),
                            ).join(pl.read_parquet("innweekstartsum.parquet"),
                                   on=["inn_id","week"],
                                   how="left",
                            )
#                            ).with_columns(                              
#                                pl.col("week").cast(pl.Int8),
#                                np.log1p(pl.col("target")),
#                            )
innweeks0

innweeks0 = innweeks0.with_columns(
                      #pl.col("sign").rolling_sum(window_size=53,min_periods=1).cast(pl.Int8).shift(0).over("inn_id").alias("inactive53"),
                      #(pl.col("sign").cum_sum()/(pl.col("week")+1)).shift(0).over("inn_id").alias("active"),

                      cs.by_name(cols).shift(0).over("inn_id"),#.name.suffix("_01"),
                      cs.by_name(cols).shift(1).over("inn_id").name.suffix("_02"),
                      cs.by_name(cols).shift(2).over("inn_id").name.suffix("_03"),
                      cs.by_name(cols).shift(3).over("inn_id").name.suffix("_04"),
                      cs.by_name(cols).shift(4).over("inn_id").name.suffix("_05"),
                      cs.by_name(cols).shift(5).over("inn_id").name.suffix("_06"),
                      cs.by_name(cols).shift(6).over("inn_id").name.suffix("_07"),
                      cs.by_name(cols).shift(7).over("inn_id").name.suffix("_08"),
                      cs.by_name(cols).shift(8).over("inn_id").name.suffix("_09"),
                      cs.by_name(cols).shift(9).over("inn_id").name.suffix("_10"),
                      cs.by_name(cols).shift(10).over("inn_id").name.suffix("_11"),
                      cs.by_name(cols).shift(11).over("inn_id").name.suffix("_12"),
                      cs.by_name(cols).shift(12).over("inn_id").name.suffix("_13"),
                      cs.by_name(cols).shift(13).over("inn_id").name.suffix("_14"),
                      cs.by_name(cols).shift(14).over("inn_id").name.suffix("_15"),
                      cs.by_name(cols).shift(15).over("inn_id").name.suffix("_16"),
                      #cs.by_name(cols).shift(16).over("inn_id").name.suffix("_17"),
                      #cs.by_name(cols).shift(17).over("inn_id").name.suffix("_18"),
                      #cs.by_name(cols).shift(18).over("inn_id").name.suffix("_19"),
                      #cs.by_name(cols).shift(19).over("inn_id").name.suffix("_20"),
                      #cs.by_name(cols).shift(20).over("inn_id").name.suffix("_21"),
                      #cs.by_name(cols).shift(21).over("inn_id").name.suffix("_22"),
                      #cs.by_name(cols).shift(22).over("inn_id").name.suffix("_23"),
                      #cs.by_name(cols).shift(23).over("inn_id").name.suffix("_24"),
                      #cs.by_name(cols).shift(24).over("inn_id").name.suffix("_25"),
                      #cs.by_name(cols).shift(25).over("inn_id").name.suffix("_26"),
                      #cs.by_name(cols).shift(26).over("inn_id").name.suffix("_27"),

                      #cs.by_name(cols).shift(51).over("inn_id").name.suffix("_52"),
                      #cs.by_name(cols).shift(52).over("inn_id").name.suffix("_53"),
                      #pl.col("target_").shift(0).over("inn_id").alias("imax1_"),
                      #pl.col("target_").shift(1).over("inn_id").alias("imax2_"),
                      #pl.col("target_").shift(2).over("inn_id").alias("imax3_"),
                      #pl.col("target_").shift(3).over("inn_id").alias("imax4_"),
                      #pl.col("target_").shift(4).over("inn_id").alias("imax5_"),
                      #pl.col("target_").shift(5).over("inn_id").alias("imax6_"),
                      #pl.col("target_").shift(6).over("inn_id").alias("imax7_"),
                      #pl.col("target_").shift(7).over("inn_id").alias("imax8_"),
                      #pl.col("target_").shift(8).over("inn_id").alias("imax9_"),
                      #pl.col("target_").shift(9).over("inn_id").alias("imax10_"),
                      #pl.col("target_").shift(10).over("inn_id").alias("imax11_"),
                      #pl.col("target_").shift(11).over("inn_id").alias("imax12_"),
                      #pl.col("target_").shift(12).over("inn_id").alias("imax13_"),
                      #pl.col("target_").shift(13).over("inn_id").alias("imax14_"),
                      #pl.col("target_").shift(14).over("inn_id").alias("imax15_"),
                      #pl.col("target_").shift(15).over("inn_id").alias("imax16_"),
                      #pl.col("target_").shift(16).over("inn_id").alias("imax17_"),
                      #pl.col("target_").shift(17).over("inn_id").alias("imax18_"),
                      #pl.col("target_").shift(18).over("inn_id").alias("imax19_"),
                      #pl.col("target_").shift(19).over("inn_id").alias("imax20_"),
                      #pl.col("target_").shift(20).over("inn_id").alias("imax21_"),
                      #pl.col("target_").shift(21).over("inn_id").alias("imax22_"),
                      #pl.col("target_").shift(22).over("inn_id").alias("imax23_"),
                      #pl.col("target_").shift(23).over("inn_id").alias("imax24_"),
                      #pl.col("target_").shift(24).over("inn_id").alias("imax25_"),
                      #pl.col("target_").shift(25).over("inn_id").alias("imax26_"),
                      #pl.col("target_").shift(26).over("inn_id").alias("imax27_"),
                      #pl.col("target_").shift(27).over("inn_id").alias("imax28_"),
                      #pl.col("target_").shift(28).over("inn_id").alias("imax29_"),
                      #pl.col("target_").shift(29).over("inn_id").alias("imax30_"),
                      #pl.col("target_").shift(30).over("inn_id").alias("imax31_"),
                      #pl.col("target_").shift(31).over("inn_id").alias("imax32_"),
                      #pl.col("target_").shift(32).over("inn_id").alias("imax33_"),
                      #pl.col("target_").shift(33).over("inn_id").alias("imax34_"),
                      #pl.col("target_").shift(34).over("inn_id").alias("imax35_"),
                      #pl.col("target_").shift(35).over("inn_id").alias("imax36_"),
                      #pl.col("target_").shift(36).over("inn_id").alias("imax37_"),
                      #pl.col("target_").shift(37).over("inn_id").alias("imax38_"),
                      #pl.col("target_").shift(38).over("inn_id").alias("imax39_"),
                      #pl.col("target_").shift(39).over("inn_id").alias("imax40_"),
                      #pl.col("target_").shift(40).over("inn_id").alias("imax41_"),
                      #pl.col("target_").shift(41).over("inn_id").alias("imax42_"),
                      #pl.col("target_").shift(42).over("inn_id").alias("imax43_"),
                      #pl.col("target_").shift(43).over("inn_id").alias("imax44_"),
                      #pl.col("target_").shift(44).over("inn_id").alias("imax45_"),
                      #pl.col("target_").shift(45).over("inn_id").alias("imax46_"),
                      #pl.col("target_").shift(46).over("inn_id").alias("imax47_"),
                      #pl.col("target_").shift(47).over("inn_id").alias("imax48_"),
                      #pl.col("target_").shift(48).over("inn_id").alias("imax49_"),
                      #pl.col("target_").shift(49).over("inn_id").alias("imax50_"),
                      #pl.col("target_").shift(50).over("inn_id").alias("imax51_"),
                      #pl.col("target_").shift(51).over("inn_id").alias("imax52_"),
                      #pl.col("target_").shift(52).over("inn_id").alias("imax53_"),
                      #pl.col("target_").shift(53).over("inn_id").alias("imax54_"),
    
                      #pl.col("target").shift(0).over("inn_id").alias("imax1"),
                      #pl.col("target").shift(1).over("inn_id").alias("imax2"),
                      #pl.col("target").shift(2).over("inn_id").alias("imax3"),
                      #pl.col("target").shift(3).over("inn_id").alias("imax4"),
                      #pl.col("target").shift(4).over("inn_id").alias("imax5"),
                      #pl.col("target").shift(5).over("inn_id").alias("imax6"),
                      #pl.col("target").shift(6).over("inn_id").alias("imax7"),
                      #pl.col("target").shift(7).over("inn_id").alias("imax8"),
                      #pl.col("target").shift(8).over("inn_id").alias("imax9"),
                      #pl.col("target").shift(9).over("inn_id").alias("imax10"),
                      #pl.col("target").shift(10).over("inn_id").alias("imax11"),
                      #pl.col("target").shift(11).over("inn_id").alias("imax12"),
                      #pl.col("target").shift(12).over("inn_id").alias("imax13"),
                      #pl.col("target").shift(13).over("inn_id").alias("imax14"),
                      #pl.col("target").shift(14).over("inn_id").alias("imax15"),
                      #pl.col("target").shift(15).over("inn_id").alias("imax16"),
                      #pl.col("target").shift(16).over("inn_id").alias("imax17"),
                      #pl.col("target").shift(17).over("inn_id").alias("imax18"),
                      #pl.col("target").shift(18).over("inn_id").alias("imax19"),
                      #pl.col("target").shift(19).over("inn_id").alias("imax20"),
                      #pl.col("target").shift(20).over("inn_id").alias("imax21"),
                      #pl.col("target").shift(21).over("inn_id").alias("imax22"),
                      #pl.col("target").shift(22).over("inn_id").alias("imax23"),
                      #pl.col("target").shift(23).over("inn_id").alias("imax24"),
                      #pl.col("target").shift(24).over("inn_id").alias("imax25"),
                      #pl.col("target").shift(25).over("inn_id").alias("imax26"),
                      #pl.col("target").shift(26).over("inn_id").alias("imax27"),
                      #pl.col("target").shift(27).over("inn_id").alias("imax28"),
                      #pl.col("target").shift(28).over("inn_id").alias("imax29"),
                      #pl.col("target").shift(29).over("inn_id").alias("imax30"),
                      #pl.col("target").shift(30).over("inn_id").alias("imax31"),
                      #pl.col("target").shift(31).over("inn_id").alias("imax32"),
                      #pl.col("target").shift(32).over("inn_id").alias("imax33"),
                      #pl.col("target").shift(33).over("inn_id").alias("imax34"),
                      #pl.col("target").shift(34).over("inn_id").alias("imax35"),
                      #pl.col("target").shift(35).over("inn_id").alias("imax36"),
                      #pl.col("target").shift(36).over("inn_id").alias("imax37"),
                      #pl.col("target").shift(37).over("inn_id").alias("imax38"),
                      #pl.col("target").shift(38).over("inn_id").alias("imax39"),
                      #pl.col("target").shift(39).over("inn_id").alias("imax40"),
                      #pl.col("target").shift(40).over("inn_id").alias("imax41"),
                      #pl.col("target").shift(41).over("inn_id").alias("imax42"),
                      #pl.col("target").shift(42).over("inn_id").alias("imax43"),
                      #pl.col("target").shift(43).over("inn_id").alias("imax44"),
                      #pl.col("target").shift(44).over("inn_id").alias("imax45"),
                      #pl.col("target").shift(45).over("inn_id").alias("imax46"),
                      #pl.col("target").shift(46).over("inn_id").alias("imax47"),
                      #pl.col("target").shift(47).over("inn_id").alias("imax48"),
                      #pl.col("target").shift(48).over("inn_id").alias("imax49"),
                      #pl.col("target").shift(49).over("inn_id").alias("imax50"),
                      #pl.col("target").shift(50).over("inn_id").alias("imax51"),
                      #pl.col("target").shift(51).over("inn_id").alias("imax52"),
                      #pl.col("target").shift(52).over("inn_id").alias("imax53"),
                      #pl.col("target").shift(53).over("inn_id").alias("imax54"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      #pl.col("target").rolling_max(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imax"),
                      #pl.col("target").rolling_min(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imin"),
                      #pl.col("target_").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("imean"),
                      #pl.col("target_").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian"),

                      #pl.col("target_").rolling_mean(window_size=2,min_periods=1).shift(0).over("inn_id").log1p().alias("imean2"),
                      #pl.col("target_").rolling_median(window_size=2,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian2"),
    
                      #pl.col("target_").rolling_mean(window_size=3,min_periods=1).shift(0).over("inn_id").log1p().alias("imean3"),
                      #pl.col("target_").rolling_median(window_size=3,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian3"),
    
                      #pl.col("target_").rolling_mean(window_size=4,min_periods=1).shift(0).over("inn_id").log1p().alias("imean4"),
                      #pl.col("target_").rolling_median(window_size=4,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian4"),
    
                      #pl.col("target_").rolling_mean(window_size=5,min_periods=1).shift(0).over("inn_id").log1p().alias("imean5"),
                      #pl.col("target_").rolling_median(window_size=5,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian5"),
    
                      #pl.col("target_").rolling_mean(window_size=13,min_periods=1).shift(0).over("inn_id").log1p().alias("imean13"),
                      #pl.col("target_").rolling_median(window_size=13,min_periods=1).shift(0).over("inn_id").log1p().alias("imedian13"),
    
                      #pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").alias("mean53"),
                      #pl.col("target").rolling_mean(window_size=13,min_periods=1).shift(0).over("inn_id").alias("mean13"),
                      #pl.col("target").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").alias("median53"),
                      #pl.col("target").rolling_median(window_size=13,min_periods=1).shift(0).over("inn_id").alias("median13"),
                      #pl.col("out_vtb_trns_amount").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imean_"),
                      #pl.col("out_vtb_trns_amount").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imedian_"),
        
                      #pl.col("target_").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq095"),
                      #pl.col("target_").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq090"),
                      #pl.col("target_").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq085"),
                      #pl.col("target_").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq080"),
                      #pl.col("target_").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq075"),
                      #pl.col("target_").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq070"),
                      #pl.col("target_").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq065"),
                      #pl.col("target_").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq060"),
                      #pl.col("target_").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq055"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq045"),
                      #pl.col("target_").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq040"),
                      #pl.col("target_").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq035"),
                      #pl.col("target_").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq030"),
                      #pl.col("target_").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq025"),
                      #pl.col("target_").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq020"),
                      #pl.col("target_").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq015"),
                      #pl.col("target_").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq010"),
                      #pl.col("target_").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(0).over("inn_id").log1p().alias("iq005"),
    
                      #pl.col("target_").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imean_"),
                      #pl.col("target_").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imedian_"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq095_"),
                      #pl.col("target_").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq090_"),
                      #pl.col("target_").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq085_"),
                      #pl.col("target_").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq080_"),
                      #pl.col("target_").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq075_"),
                      #pl.col("target_").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq070_"),
                      #pl.col("target_").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq065_"),
                      #pl.col("target_").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq060_"),
                      #pl.col("target_").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq055_"),
    
                      #pl.col("target_").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq045_"),
                      #pl.col("target_").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq040_"),
                      #pl.col("target_").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq035_"),
                      #pl.col("target_").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq030_"),
                      #pl.col("target_").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq025_"),
                      #pl.col("target_").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq020_"),
                      #pl.col("target_").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq015_"),
                      #pl.col("target_").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq010_"),
                      #pl.col("target_").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq005_"),
                     )
innweeks0 = innweeks0.group_by("inn_id").last().drop(["week","target"])
innweeks0

CPU times: user 50.5 s, sys: 16.1 s, total: 1min 6s
Wall time: 50.8 s


inn_id,sign,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,in_vtb_trns_count,in_vtb_trns_amount,in_vtb_date,in_vtb_cinn,in_vtb_trns_amount_log,in_vtbip_trns_count,in_vtbip_trns_amount,in_vtbip_cinn,cum,cumsum,cumsum_right,out_vtb_trns_count_02,out_vtb_trns_amount_02,out_vtb_date_02,out_vtb_cinn_02,out_vtb_trns_amount_log_02,out_vtbip_trns_count_02,out_vtbip_trns_amount_02,out_vtbip_cinn_02,in_vtb_trns_count_02,in_vtb_trns_amount_02,in_vtb_date_02,in_vtb_cinn_02,in_vtb_trns_amount_log_02,in_vtbip_trns_count_02,in_vtbip_trns_amount_02,in_vtbip_cinn_02,…,in_vtbip_trns_amount_14,in_vtbip_cinn_14,cum_14,out_vtb_trns_count_15,out_vtb_trns_amount_15,out_vtb_date_15,out_vtb_cinn_15,out_vtb_trns_amount_log_15,out_vtbip_trns_count_15,out_vtbip_trns_amount_15,out_vtbip_cinn_15,in_vtb_trns_count_15,in_vtb_trns_amount_15,in_vtb_date_15,in_vtb_cinn_15,in_vtb_trns_amount_log_15,in_vtbip_trns_count_15,in_vtbip_trns_amount_15,in_vtbip_cinn_15,cum_15,out_vtb_trns_count_16,out_vtb_trns_amount_16,out_vtb_date_16,out_vtb_cinn_16,out_vtb_trns_amount_log_16,out_vtbip_trns_count_16,out_vtbip_trns_amount_16,out_vtbip_cinn_16,in_vtb_trns_count_16,in_vtb_trns_amount_16,in_vtb_date_16,in_vtb_cinn_16,in_vtb_trns_amount_log_16,in_vtbip_trns_count_16,in_vtbip_trns_amount_16,in_vtbip_cinn_16,cum_16
str,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,f32,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,…,u8,u8,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32,u16,f32,u8,u16,f32,u8,u8,u8,u16,f32,u8,u16,f32,u8,u8,u8,f32
"""inn1586477""",1.0,21,2.678225e6,4,20,14.800665,0,0,0,6,258644.015625,2,6,12.463212,84,181,84,-2.419581e6,-1.1909e8,30704.0,20,5510644.5,3,18,15.522192,25,8,28,5,681574.625,2,5,13.432162,50,3,50,…,2,84,541161.125,11,1.722059e6,4,11,14.359032,69,6,69,4,6.127569e6,2,4,15.628309,0,0,0,4.40551e6,32,4493350.5,4,25,15.31811,31,25,30,9,4.737917e6,4,7,15.371108,0,0,0,244566.140625
"""inn5163055""",1.0,40,1579574.5,6,32,14.272667,70,152,79,14,1.4029e6,5,4,14.154047,18,51,63,-176683.125,2.7280318e7,2.8280608e7,39,2974688.5,4,23,14.90565,58,230,77,25,1.935008e6,6,5,14.475622,0,0,0,…,0,0,1507844.5,32,650406.875,5,21,13.385355,103,44,97,25,2.2355e6,6,11,14.619997,61,146,92,1.5851e6,32,386267.28125,4,21,12.864287,55,43,48,12,1.1590e6,5,4,13.963039,0,0,0,772698.0
"""inn3463310""",1.0,12,393302.15625,3,7,12.882336,21,87,36,11,462625.75,6,2,13.044676,23,23,127,69323.59375,2.0935584e7,2.1107e7,13,427233.28125,2,8,12.965088,19,22,31,13,420837.65625,5,1,12.950005,0,0,0,…,0,0,208302.421875,5,101160.640625,2,5,11.524475,50,79,50,3,21731.712891,3,1,9.986574,0,0,0,-79428.929688,3,46629.058594,1,3,10.750001,0,0,0,6,166664.203125,3,1,12.023743,0,0,0,120035.140625
"""inn3981633""",1.0,32,527427.4375,7,23,13.175769,71,56,88,7,830265.1875,4,3,13.629501,109,252,84,302837.75,-2.0275898e7,8.426662e6,33,223402.03125,6,22,12.316732,84,64,104,6,228991.25,2,3,12.341443,0,0,0,…,255,254,241802.5,35,349537.03125,6,31,12.764368,94,99,98,8,60112.601562,5,3,11.003991,0,0,0,-289424.4375,37,1.3877e6,7,23,14.143137,41,1,66,11,1.0549e6,3,8,13.868944,23,13,31,-332783.59375
"""inn2901266""",1.0,9,184503.53125,4,3,12.125429,226,247,169,4,1.7840e6,2,3,14.394383,0,0,0,1.5995e6,1.7732134e7,1.7137736e7,11,292393.84375,4,5,12.58586,46,16,50,2,775238.125,1,1,13.560926,0,0,0,…,0,0,-2126.645996,7,282563.1875,4,4,12.551661,145,106,63,2,1242449.5,1,2,14.032597,0,0,0,959886.25,9,312263.5625,4,5,12.651606,141,47,50,0,0.0,0,0,0.0,0,0,0,-312263.5625
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn4209570""",1.0,14,1.5856e6,5,8,14.276444,72,134,95,18,1.0189e6,5,10,13.834283,56,39,76,-566604.125,-1.2215e8,279608.0,13,1861307.5,4,10,14.43679,19,44,25,13,672628.6875,5,7,13.41895,39,58,36,…,44,127,-1.6312e6,14,3.0435e6,6,10,14.92853,54,48,50,23,1.1031e6,6,12,13.9136

In [42]:
X0=pl.read_csv("data/sample_submit.csv"
           ).with_columns( pl.col("week").cast(pl.Int8)
            ).join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        ).join(innweeks0,on="inn_id",
                         how="left"
                  #).join(weekclass.with_columns(pl.col("class_nweek")),
                  #       left_on=["inn_id","week"],
                  #       right_on=["class_doc_payer_inn","class_nweek"],
                  #       how="left"
                  #).join(weekclass.with_columns(pl.col("class_nweek")+1),
                  #       left_on=["inn_id","week"],
                  #       right_on=["class_doc_payer_inn","class_nweek"],
                  #       suffix="02",
                  #       how="left"
                  #).join(weekclass.with_columns(pl.col("class_nweek")+2),
                  #       left_on=["inn_id","week"],
                  #       right_on=["class_doc_payer_inn","class_nweek"],
                  #       suffix="03",
                  #       how="left"
                  #).join(weekclass.with_columns(pl.col("class_nweek")+3),
                  #       left_on=["inn_id","week"],
                  #       right_on=["class_doc_payer_inn","class_nweek"],
                  #       suffix="04",
                  #       how="left"
                               
                        ).select(X_train.columns).to_pandas()
X0

,inn_id,week,out_vtb_trns_count,out_vtb_trns_amount,out_vtb_date,out_vtb_cinn,out_vtb_trns_amount_log,out_vtbip_trns_count,out_vtbip_trns_amount,out_vtbip_cinn,...,w7,w7__3,w7__2,w7__1,ipul,id_region,main_okved_group,diff_datopen_report_date_flg,diff_datopen_report_date_flg_min,report_date_min
0,inn1000051,106,7,7430786.0,3,5,15.821142,36,0,50,...,5,4.0,5.0,5.0,0,45,68,96.0,48.0,2023-10-01
1,inn1000051,107,7,7430786.0,3,5,15.821142,36,0,50,...,5,5.0,4.0,5.0,0,45,68,96.0,48.0,2023-10-01
2,inn1000051,108,7,7430786.0,3,5,15.821142,36,0,50,...,5,5.0,5.0,5.0,0,45,68,96.0,48.0,2023-10-01
3,inn1000051,109,7,7430786.0,3,5,15.821142,36,0,50,...,5,5.0,5.0,5.0,0,45,68,96.0,48.0,2023-10-01
4,inn1000051,110,7,7430786.0,3,5,15.821142,36,0,50,...,5,5.0,5.0,5.0,0,45,68,96.0,48.0,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623551,inn999886,113,44,19776084.0,5,26,16.799984,34,2,39,...,5,5.0,5.0,5.0,1,22,45,192.0,192.0,2023-10-01
623552,inn999886,114,44,19776084.0,5,26,16.799984,34,2,39,...,5,NaN,NaN,NaN,1,22,45,192.0,192.0,2023-10-01
623553,inn999886,115,44,19776084.0,5,26,16.799984,34,2,39,...,5,5.0,5.0,5.0,1,22,45,192.0,192.0,2023-10-01
623554,inn999886,116,44,19776084.0,5,26,16.799984,34,2,39,...,5,5.0,5.0,5.0,1,22,45,192.0,192.0,2023-10-01


In [43]:
#model.load_model("model0.15696200728416443_{'learn': {'RMSE': 2.1840243465326776}, 'validation': {'RMSE': 2.4295050367945095}}.cbm")
model

In [44]:
y_test

array([[15.044251, 14.610071, 11.487807, 15.821142],
       [15.053899, 15.62723 , 15.110888, 15.626517],
       [12.78503 ,  9.141981, 10.774144, 10.328082],
       ...,
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [16.11803 , 16.202227, 16.094133, 16.255236],
       [17.042799, 16.736929, 17.025045, 16.799984]], dtype=float32)

In [45]:
#np.expm1( np.clip(model.predict(X_test), 0, np.inf) )
np.clip(model.predict(X_test), 0, np.inf)

array([[10.46699448, 10.08585621, 11.20028876, 10.62694326],
       [15.28367984, 15.34115784, 15.29024239, 15.29999766],
       [12.10170316, 12.05119577, 12.19806385, 12.21971521],
       ...,
       [12.79900814, 12.95557067, 12.49014559, 12.25750457],
       [16.36429862, 16.38619061, 16.40291294, 16.24429263],
       [16.40034364, 16.37511178, 16.36456832, 16.2641361 ]])

In [46]:
np.expm1( np.clip(model.predict(X0), 0, np.inf) )

array([[  207748.45389256,    97152.02869762,   255401.20002551,
          278390.20761765],
       [  203981.11761573,   101743.20180649,   226128.79552606,
          341148.63238516],
       [  163530.50898716,    92610.70863671,   288175.31993816,
          381979.66103414],
       ...,
       [17465351.64989183, 17476540.30454716, 17993709.60306681,
        16571117.67185342],
       [18470626.90008631, 17812940.31858774, 17992784.9384098 ,
        16404191.1332875 ],
       [17126004.24112172, 17259900.2885621 , 16583029.54170805,
        15163418.82862594]])

In [47]:
Xp=pl.read_csv("data/sample_submit.csv"
           ).to_pandas()
pp=np.expm1( np.clip(model.predict(X0), 0, np.inf) )
Xp["predict"]=np.ravel(np.concatenate((pp, pp, pp), axis=1)[::12,])
#Xp["predict1"]=pp[:,1]
#Xp["predict2"]=pp[:,2]
#Xp["predict3"]=pp[:,3]
Xp.to_csv("pred_016bb27_d8_103-3000-16.csv")
Xp

,inn_id,week,predict
0,inn1000051,106,2.077485e+05
1,inn1000051,107,9.715203e+04
2,inn1000051,108,2.554012e+05
3,inn1000051,109,2.783902e+05
4,inn1000051,110,2.077485e+05
...,...,...,...
623551,inn999886,113,1.651081e+07
623552,inn999886,114,1.654313e+07
623553,inn999886,115,1.589562e+07
623554,inn999886,116,1.678738e+07


In [48]:
#innweeks.filter(pl.col("inn_id")=="inn999737")